# Install libraries

In [15]:
import os
import matplotlib.pyplot as plt
from netCDF4 import Dataset as netcdf_dataset
import numpy as np

import cartopy
from cartopy import config
import cartopy.crs as ccrs
import cartopy.feature as cfeature


# Load & examine dataset

In [16]:
fname = 'gdas.t00z.sfluxgrbf000..nc'

In [17]:
dataset = netcdf_dataset(fname)

In [18]:
dataset

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    GRIB_edition: 2
    GRIB_centre: kwbc
    GRIB_centreDescription: US National Weather Service - NCEP
    GRIB_subCentre: 0
    Conventions: CF-1.7
    institution: US National Weather Service - NCEP
    history: 2023-07-06T14:28 GRIB to CDM+CF via cfgrib-0.9.10.1/ecCodes-2.26.0 with {"source": "data/gdas.t00z.sfluxgrbf000/gdas.t00z.sfluxgrbf000.grib2", "filter_by_keys": {"typeOfLevel": "hybrid"}, "encode_cf": ["parameter", "time", "geography", "vertical"]}
    dimensions(sizes): latitude(1536), longitude(3072)
    variables(dimensions): int64 time(), float64 step(), float64 hybrid(), float64 latitude(latitude), float64 longitude(longitude), float64 valid_time(), float32 gh(latitude, longitude), float32 t(latitude, longitude), float32 q(latitude, longitude), float32 u(latitude, longitude), float32 v(latitude, longitude)
    groups: 

# Different variables in nc file
- 'time': This variable represents the initial time of forecast in seconds since 1970-01-01.
- 'step': This variable represents the time since forecast_reference_time in hours.
- 'hybrid': This variable represents the hybrid level.
- 'latitude': This variable represents the latitude in degrees north.
- longitude: This variable represents the longitude in degrees east.
- 'valid_time': This variable represents the time in seconds since 1970-01-01.
- 'gh': This variable represents the geopotential height in gpm.
- 't': This variable represents the temperature in K.
- 'q': This variable represents the specific humidity in kg kg**-1.
- 'u': This variable represents the U component of wind in m s**-1.
- 'v': This variable represents the V component of wind in m s**-1.

In [20]:
dataset.variables

{'time': <class 'netCDF4._netCDF4.Variable'>
 int64 time()
     long_name: initial time of forecast
     standard_name: forecast_reference_time
     units: seconds since 1970-01-01
     calendar: proleptic_gregorian
 unlimited dimensions: 
 current shape = ()
 filling on, default _FillValue of -9223372036854775806 used,
 'step': <class 'netCDF4._netCDF4.Variable'>
 float64 step()
     _FillValue: nan
     long_name: time since forecast_reference_time
     standard_name: forecast_period
     units: hours
 unlimited dimensions: 
 current shape = ()
 filling on,
 'hybrid': <class 'netCDF4._netCDF4.Variable'>
 float64 hybrid()
     _FillValue: nan
     long_name: hybrid level
     units: 1
     positive: down
     standard_name: atmosphere_hybrid_sigma_pressure_coordinate
 unlimited dimensions: 
 current shape = ()
 filling on,
 'latitude': <class 'netCDF4._netCDF4.Variable'>
 float64 latitude(latitude)
     _FillValue: nan
     units: degrees_north
     standard_name: latitude
     long_n

# Prep & examine variables

In [21]:
u_wind = dataset.variables['u'][:]
v_wind = dataset.variables['v'][:]

In [22]:
lats = dataset.variables['latitude'][:]
lons = dataset.variables['longitude'][:]

In [23]:
lats

masked_array(data=[ 89.91032453,  89.79415739,  89.67730421, ...,
                   -89.67730421, -89.79415739, -89.91032453],
             mask=False,
       fill_value=1e+20)

In [24]:
v_wind

masked_array(
  data=[[ 2.88,  2.87,  2.85, ...,  2.93,  2.91,  2.9 ],
        [ 2.99,  2.98,  2.96, ...,  3.04,  3.02,  3.01],
        [ 2.94,  2.92,  2.91, ...,  2.98,  2.97,  2.95],
        ...,
        [-4.72, -4.72, -4.72, ..., -4.71, -4.71, -4.71],
        [-4.75, -4.75, -4.76, ..., -4.75, -4.75, -4.75],
        [-4.58, -4.58, -4.59, ..., -4.58, -4.58, -4.58]],
  mask=False,
  fill_value=1e+20,
  dtype=float32)

# Calculate variables from other vars

In [25]:
wind_speed = np.sqrt(u_wind**2 + v_wind**2)
ws_direction = np.arctan2(v_wind,u_wind)
ws_daily_avg = np.nanmean(wind_speed, axis=0)
u_daily_avg = np.nanmean(u_wind, axis=0)
v_daily_avg = np.nanmean(v_wind, axis=0)
ws_daily_avg_direction = np.arctan2(v_daily_avg, u_daily_avg)


In [26]:
ws_daily_avg_direction

masked_array(data=[0.8478495, 0.8327088, 0.8086573, ..., 0.9755575,
                   0.9411184, 0.8905726],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [27]:
wind_speed

masked_array(
  data=[[8.164802551269531, 8.170642852783203, 8.173009872436523, ...,
         8.163902282714844, 8.166088104248047, 8.171878814697266],
        [8.260150909423828, 8.256536483764648, 8.258674621582031, ...,
         8.259781837463379, 8.261749267578125, 8.258098602294922],
        [8.214164733886719, 8.216373443603516, 8.212825775146484, ...,
         8.20992660522461, 8.215624809265137, 8.208415985107422],
        ...,
        [4.724668979644775, 4.724235534667969, 4.723822593688965, ...,
         4.716110706329346, 4.715612411499023, 4.71513557434082],
        [4.7523674964904785, 4.752062797546387, 4.762058734893799, ...,
         4.753409385681152, 4.7530412673950195, 4.752694129943848],
        [4.581320762634277, 4.581091403961182, 4.590882301330566, ...,
         4.582139015197754, 4.581844329833984, 4.581571578979492]],
  mask=[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, 

In [28]:
lats = dataset.variables['latitude'][:]
lons = dataset.variables['longitude'][:]

# Create color maps

In [ ]:
my_cmap = plt.colormaps['twilight']
my_cmap2 = plt.colormaps['twilight_shifted']

# Plot maps

In [ ]:
plt.figure(figsize=(100,50))
ax = plt.axes(projection=ccrs.PlateCarree())

plt.contourf(lons, lats, wind_speed, 60,
             transform=ccrs.PlateCarree())

ax.coastlines()

plt.show()

In [ ]:
plt.figure(figsize=(10,5))
ax = plt.axes(projection=ccrs.PlateCarree())

plt.contourf(lons, lats, wind_speed, 60,
             transform=ccrs.PlateCarree(),cmap=my_cmap)

ax.coastlines()

plt.show()

In [ ]:
plt.figure(figsize=(100,50))
ax = plt.axes(projection=ccrs.PlateCarree())

plt.contourf(lons, lats, wind_speed, 60,
             transform=ccrs.PlateCarree(),cmap=my_cmap2)

ax.coastlines()

plt.show()

In [ ]:
# Set the figure size, projection, and extent
fig = plt.figure(figsize=(100,50))
ax = plt.axes(projection=ccrs.Robinson())
ax.set_global()
ax.coastlines(resolution="110m",linewidth=1)
ax.gridlines(linestyle='--',color='black')

# Plot windspeed:
# Set contour levels, then draw the plot and a colorbar
clevs = np.arange(0,19,1)
plt.contourf(lon, lat, wind_speed, clevs, transform=ccrs.PlateCarree(),cmap=my_cmap)
plt.title('Wind Speed', size=30)
cb = plt.colorbar(ax=ax, orientation="vertical", pad=0.02, aspect=16, shrink=0.8)
cb.set_label('m/s',size=12,rotation=0,labelpad=15)
cb.ax.tick_params(labelsize=10)
     

# Plot maps with arrows

In [ ]:
# Set the figure size, projection, and extent
fig = plt.figure(figsize=(9,5))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([22, 40, 43, 53], crs=ccrs.PlateCarree())
ax.coastlines(resolution="50m",linewidth=1)


# Plot windspeed
clevs = np.arange(0,14.5,1)
plt.contourf(lon, lat, wind_speed, clevs, transform=ccrs.PlateCarree(),cmap=my_cmap)
cb = plt.colorbar(ax=ax, orientation="vertical", pad=0.02, aspect=16, shrink=0.8)
cb.set_label('m/s',size=14,rotation=0,labelpad=15)
cb.ax.tick_params(labelsize=10)
# Overlay wind vectors
qv = plt.quiver(lon, lat, u_wind, v_wind, scale=1000, color='k')
plt.show()